# map2loop:  Extracting fault-fault and fault-strat relationship tables from map2model output

In [1]:
import os
import numpy as np
import pandas as pd
#os.chdir('../')

print(os.getcwd())

C:\Users\00073294\Dropbox\1_Jupyter_notebooks\map2loop\notebooks


In [2]:
uf=open('../test_data2/graph/output/unit-fault-intersection.txt','r')
contents =uf.readlines()
uf.close()

all_faults={}
unique_list = [] 

for row in contents:
    row=row.replace("\n","").split("{")
    unit=row[0].split(',')
    faults=row[1].replace("}","").replace(" ","").split(",")

    for f in faults:
        if f not in unique_list: 
            unique_list.append(f) 

uf=open('../test_data2/output/unit-fault-relationships.csv','w')
uf.write('code,'+str(unique_list).replace("[","").replace("'","").replace("]","").replace(" ","")+'\n')
for row in contents:
    row=row.replace("\n","").split("{")
    unit=row[0].split(',')
    faults=row[1].replace("}","").replace(" ","").split(",")
    ostr=str(unit[1]).replace(" ","")
    for ul in unique_list:
        out=[item for item in faults if ul in item]
        if(len(out)>0):
            ostr=ostr+",1"
        else:
            ostr=ostr+",0" 
    uf.write(ostr+"\n")
uf.close()



In [3]:
summary=pd.read_csv('../test_data2/tmp/hams2_all_sorts.csv')
summary.set_index("code", inplace=True)

uf_rel=pd.read_csv('../test_data2/output/unit-fault-relationships.csv')

f=open('../test_data2/tmp/hams2_groups.csv',"r")
groups =f.readlines()
f.close
ngroups=groups[0].split(" ")
ngroups=int(ngroups[1])


uf_array=uf_rel.to_numpy()
gf_array=np.zeros((ngroups,uf_array.shape[1]),dtype='U25')
print(gf_array.shape)
display(uf_array)

(6, 105)


array([['A-FO-od', 1, 1, ..., 0, 0, 0],
       ['A-FO-xo-a', 0, 0, ..., 0, 0, 0],
       ['A-FOh-xs-f', 0, 0, ..., 0, 0, 0],
       ...,
       ['P_-WYa-st', 0, 0, ..., 0, 0, 0],
       ['P_-WYd-kd', 0, 0, ..., 0, 0, 0],
       ['P_-WYm-sp', 0, 0, ..., 0, 0, 0]], dtype=object)

In [4]:
#print(summary.index.values)
for i in range(1,ngroups+1):
    #print("---------",groups[i].replace("\n",""))
    for j in range(0,len(uf_rel)):
        #print("uf_rel",uf_rel.iloc[j][0])
        if(uf_rel.iloc[j][0] in summary.index.values):
            gsummary=summary.loc[uf_rel.iloc[j][0]]
            #print("sum gp",gsummary['group'])
            if(groups[i].replace("\n","")==gsummary['group']):
                #print("yes group", groups[i].replace("\n",""),gsummary['group'])
                for k in range(1,len(uf_rel.iloc[j])):
                    if(uf_rel.iloc[j][k]==1):
                        gf_array[i-1,k]='1'
                        #print(contents[i].replace("\n",""),au,uf_rel.columns[k],uf_rel.iloc[j][k])
                    else:
                        continue
                        #print(contents[i].replace("\n",""),au,uf_rel.columns[k],'nope')
            else:
                continue
                #print("not group", groups[i].replace("\n",""),gsummary['group'])
        else:
            continue
            #print("not found",uf_rel.iloc[j][0])
#display(gf_array)

In [5]:
ug=open('../test_data2/output/group-fault-relationships.csv','w')
ug.write('group')
for k in range(1,len(uf_rel.iloc[0])):
    ug.write(','+uf_rel.columns[k])
ug.write("\n")
for i in range(1,ngroups+1):
    ug.write(groups[i].replace("\n",""))
    for k in range(1,len(uf_rel.iloc[0])):
        if(gf_array[i-1,k]=='1'):
            ug.write(',1')
        else:
            ug.write(',0')
    ug.write("\n")

ug.close()
                   

In [6]:
uf=open('../test_data2/graph/output/fault-fault-intersection.txt','r')
contents =uf.readlines()
uf.close()

unique_list = [] 

for row in contents:
    row=row.replace("\n","").split("{")
    fault_1o=row[0].split(',')
    fault_1o=fault_1o[1]
    faults_2o=row[1].replace("(","").replace(")","").replace("}","").split(",")

    if fault_1o.replace(" ","") not in unique_list: 
            unique_list.append(fault_1o.replace(" ","")) 
    for i in range (0,len(faults_2o),3):
        if faults_2o[i].replace(" ","") not in unique_list: 
                unique_list.append(faults_2o[i].replace(" ","")) 
#display(unique_list) 

ff=open('../test_data2/output/fault-fault-relationships.csv','w')
ff.write('fault_id')
for i in range (0,len(unique_list)):
    ff.write(','+unique_list[i])
ff.write('\n')

for i in range(0,len(unique_list)): #loop thorugh rows
    ff.write(unique_list[i]) 
    found=False
    #for j in range(0,len(unique_list)):
    for row in contents: #loop thorugh known intersections
        row=row.replace("\n","").split("{")
        fault_1o=row[0].split(',')
        fault_1o=fault_1o[1]
        faults_2o=row[1].replace("(","").replace(")","").replace("}","").split(",")

        if(unique_list[i].replace(" ","")==fault_1o.replace(" ","")): #correct first order fault for this row
            found=True
            for k in range(0,len(unique_list)): #loop through columns
                found2=False
                if(k==i): # no self intersections
                    ff.write(',0')
                else:
                    for f2o in range (0,len(faults_2o),3): #loop through second order faults for this row
                        if (faults_2o[f2o].replace(" ","")==unique_list[k].replace(" ","")):
                            ff.write(',1')
                            found2=True
                            break

                if(not found2 and k!=i):
                    ff.write(',0') #this is not a second order fault for this row
        if(found):
            break
    if(not found): #this fault is not a first order fault relative to another fault
        for i in range (0,len(unique_list)):
            ff.write(',0')
        
    ff.write('\n')

ff.close()